In [1]:
import glob
import pandas as pd
catp = glob.glob("train/train/cat.*.*")
dogp = glob.glob("train/train/dog.*.*")
catt = [0] * len(catp)
dogt = [1] * len(dogp)
df = pd.DataFrame({
    "path":catp + dogp,
    "target":catt + dogt
})
df

,path,target
0,train/train\cat.0.jpg,0
1,train/train\cat.1.jpg,0
2,train/train\cat.10.jpg,0
3,train/train\cat.100.jpg,0
4,train/train\cat.1000.jpg,0
...,...,...
24995,train/train\dog.9995.jpg,1
24996,train/train\dog.9996.jpg,1
24997,train/train\dog.9997.jpg,1
24998,train/train\dog.9998.jpg,1


In [2]:
from keras.applications.vgg16 import VGG16
vgg = VGG16(include_top=False, input_shape=(224, 224, 3))
vgg.summary()

Using TensorFlow backend.
c:\users\a30\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\a30\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\a30\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])


Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0    

In [3]:
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
# 如果你要設置trainable與否, 一定要compile前
for l in vgg.layers:
    l.trainable = False
x = Flatten()(vgg.output)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(2, activation="softmax")(x)
model = Model(inputs=vgg.input, outputs=x)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [4]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

In [5]:
# 進到機器學習以後, 最好不要再出現pandas, 全部轉換nparray
import numpy as np
from sklearn.model_selection import train_test_split
x = np.array(df["path"])
y = np.array(df["target"])
result = train_test_split(x, y, test_size=0.1)
x_train, x_test, y_train, y_test = result

In [6]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(22500,)
(22500,)
(2500,)
(2500,)


In [7]:
# https://github.com/keras-team/keras-applications/blob/master/keras_applications/imagenet_utils.py
from keras.preprocessing.image import load_img
from keras.applications.vgg16 import preprocess_input
def get_images(x, y, batch):
    idx = np.random.randint(0, x.shape[0], batch)
    paths = x[idx]
    
    ori_img_list = []
    img_list = []
    for p in paths:
        img = load_img(p, target_size=(224, 224)).convert("RGB")
        img = np.array(img)
        img_pre = preprocess_input(img.astype("float64"))
        ori_img_list.append(img)
        img_list.append(img_pre)
    return (np.array(ori_img_list), 
            np.array(img_list), 
            np.array(y[idx]))
        
ori, pre, target = get_images(x_train, y_train, 20)
print(ori.shape)
print(pre.shape)
print(target.shape)
# print(pre)

(20, 224, 224, 3)
(20, 224, 224, 3)
(20,)


In [8]:
TRAIN_BATCH = 20
VALIDATE_BATCH = 20

for i in range(10):
    _, imgs, targets = get_images(x_train, y_train, TRAIN_BATCH)
    train_result = model.train_on_batch(imgs, targets)
    print("[Train]:", train_result)
    _, imgs, targets = get_images(x_test, y_test, VALIDATE_BATCH)
    test_result = model.test_on_batch(imgs, targets)
    print("[Test]:", test_result)


[Train]: [21.850653, 0.3]
[Test]: [217.37036, 0.4]
[Train]: [233.87564, 0.35]
[Test]: [13.601847, 0.45]
[Train]: [13.580627, 0.65]
[Test]: [75.635704, 0.5]
[Train]: [38.388355, 0.55]
[Test]: [66.56581, 0.6]
[Train]: [106.30686, 0.3]
[Test]: [2.839174, 0.9]
[Train]: [5.346203, 0.9]
[Test]: [35.963703, 0.55]
[Train]: [48.101494, 0.6]
[Test]: [21.992657, 0.65]
[Train]: [36.926888, 0.65]
[Test]: [5.377515, 0.8]
[Train]: [11.097082, 0.65]
[Test]: [2.4820693, 0.95]
[Train]: [1.0407034, 0.95]
[Test]: [1.788139e-08, 1.0]


In [9]:
model.predict(imgs)

array([[1.0000000e+00, 0.0000000e+00],
       [9.9999964e-01, 3.8308403e-07],
       [0.0000000e+00, 1.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [3.6568742e-21, 1.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00]], dtype=float32)